In [23]:
import subprocess
from pathlib import Path
from collections import defaultdict
from multiprocessing import Pool

In [5]:
sample_names = ['R20-0148', 'R20-0127', 'R20-0131', 'R20-0026', 'R20-0158', 'R19-2905',
                'R20-0145', 'R20-0140', 'R20-0160', 'R20-0088', 'R20-0030', 'R20-0150']

In [8]:
long_reads_path = Path('/media/NGS/Nanopore_1/20200925/fastq/nanopore')
contigs_path = Path('/media/NGS/Data_Analysis/20201221_hybrid_and_denovo/hybrid_sequences_mapping/genomes')
outpath = Path('/media/NGS/Data_Analysis/20201221_hybrid_and_denovo/hybrid_sequences_mapping/lr/')

In [9]:
for sample_name in sample_names:
    query = long_reads_path/(sample_name + '.fastq')
    reference = contigs_path/(sample_name + '.fa')
    outfile = outpath/(sample_name + '.sort.bam')
    cmd = f" minimap2 -ax map-ont {reference} {query} -t 32 | samtools view -bS -@ 32 | samtools sort -@ 32 -o {outfile}"
    subprocess.run(cmd, shell=True, check=True)

In [17]:
dirpath = Path('/media/NGS/Nanopore_1/20200925/fastq/illumina')
outpath = Path('/media/NGS/Data_Analysis/20201221_hybrid_and_denovo/hybrid_sequences_mapping/sr')

pairwise = defaultdict(list)
for filepath in dirpath.iterdir():
    prefix = filepath.stem.split('_')[0]
    pairwise[prefix].append(filepath.as_posix())

In [19]:
for sample_name in sample_names:
    query = ' '.join(pairwise[sample_name])
    reference = contigs_path/(sample_name + '.fa')
    outfile = outpath/(sample_name + '.sort.bam')
    cmd = f"minimap2 -ax sr {reference} {query} -t 32 | samtools view -bS -@ 32 | samtools sort -@ 32 -o {outfile}"
    subprocess.run(cmd, shell=True, check=True)

In [25]:
dirpath = Path('/media/NGS/Data_Analysis/20201221_hybrid_and_denovo/hybrid_sequences_mapping/genomes')

with Pool(12) as p:
    try:
        for filepath in dirpath.iterdir():
            cmd = f"samtools faidx {filepath}"
            p.apply_async(subprocess.run, (cmd,), {'shell': True, 'check': True})
        p.close()
        p.join()
    except:
        p.terminate()